In [30]:
import pandas as pd
import numpy as np
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder 
from sklearn.model_selection import train_test_split
from category_encoders import HashingEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer


data = pd.read_csv('train.csv')
data.shape

(381109, 12)

In [21]:
cat_feat = ['Gender', 'Vehicle_Age','Vehicle_Damage']
num_feat = ['Age','Annual_Premium','Region_Code','Policy_Sales_Channel','Driving_License','Previously_Insured']

In [22]:
def encode():
        
    categorical_transformer = Pipeline(steps=[
            ('onehot', OneHotEncoder(handle_unknown='ignore'))])


    numeric_transformer = Pipeline(
            steps = [
                ("scaler", StandardScaler())]
        )



    preprocessor = ColumnTransformer(
        transformers=[
            ("onehot", categorical_transformer, ['Gender', 'Vehicle_Age','Vehicle_Damage']),
            ("num", numeric_transformer,['Age','Annual_Premium','Region_Code','Policy_Sales_Channel','Driving_License','Previously_Insured'])
        ]
    )

    clf = Pipeline(
        steps=[("preprocessor", preprocessor)]
    )

    return clf

In [23]:
clf = encode()
#contain all  independent variable
x=data.drop(['Response'],axis=1)  
y=data['Response']   

train = RandomUnderSampler(random_state=42, replacement=True)
x_train, y_train = train.fit_resample(data.iloc[:,0:-1], data['Response'])

xtrain,xtest,ytrain,ytest=train_test_split(x_train,y_train,test_size=.20,random_state=0)

xtrain=clf.fit_transform(xtrain)
xtest=clf.transform(xtest)

print('Preprocessed : xtrain, xtest, ytrain, ytest') 

print(xtrain.shape,xtest.shape,ytrain.shape,ytest.shape)

Preprocessed : xtrain, xtest, ytrain, ytest
(74736, 13) (18684, 13) (74736,) (18684,)


In [24]:
cat_cols = clf.named_steps['preprocessor'].transformers_[0][1] \
        .named_steps['onehot'].get_feature_names_out(cat_feat)

features_df = pd.DataFrame(xtest, columns=cat_cols.tolist()+num_feat)

In [25]:
features_df.head()

,Gender_Female,Gender_Male,Vehicle_Age_1-2 Year,Vehicle_Age_< 1 Year,Vehicle_Age_> 2 Years,Vehicle_Damage_No,Vehicle_Damage_Yes,Age,Annual_Premium,Region_Code,Policy_Sales_Channel,Driving_License,Previously_Insured
0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,-0.055672,-0.018430,0.116197,0.368863,0.038567,-0.597627
1,1.0,0.0,1.0,0.0,0.0,0.0,1.0,-1.311685,0.149445,-0.754500,-1.384062,0.038567,-0.597627
2,0.0,1.0,0.0,1.0,0.0,0.0,1.0,-1.172128,1.628837,0.116197,0.905473,0.038567,-0.597627
3,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.549234,0.309843,0.116197,-1.384062,0.038567,-0.597627
4,1.0,0.0,0.0,1.0,0.0,0.0,1.0,-0.823235,-1.592778,0.116197,0.368863,0.038567,-0.597627
